In [1]:
from TaxiFareModel.trainer import Trainer
from TaxiFareModel.data import get_data, clean_df
from sklearn import set_config
import sklearn
set_config(display='diagram')

In [5]:
df=get_data()

df=clean_df(df)

get_data 1.92


In [6]:
y_train = df["fare_amount"]
X_train = df.drop("fare_amount", axis=1)
print("shape: {}".format(X_train.shape))
print("size: {} Mb".format(X_train.memory_usage().sum() / 1e6))

shape: (8533, 7)
size: 0.546112 Mb


In [7]:
experiment='Taxi_fare_model_566'
params = dict(nrows=10000,
                  local=False,  # set to False to get data from GCP (Storage or BigQuery)
                  optimize=True,
                  estimator="xgboost",
                  mlflow=True,  # set to True to log params to mlflow
                  experiment_name=experiment,
                  pipeline_memory=None,
                  distance_type="manhattan",
                  feateng=["distance_to_center","direction","distance","time_features","geohash"])


In [8]:
trainer = Trainer(X_train, y_train, **params)

In [9]:
trainer.set_pipeline()

XGBRegressor


In [11]:
trainer.pipeline.get_params()

{'memory': None,
 'steps': [('features',
   ColumnTransformer(transformers=[('distance',
                                    Pipeline(steps=[('distancetransformer',
                                                     DistanceTransformer(distance_type='manhattan')),
                                                    ('standardscaler',
                                                     StandardScaler())]),
                                    ['pickup_latitude', 'pickup_longitude',
                                     'dropoff_latitude', 'dropoff_longitude']),
                                   ('time_features',
                                    Pipeline(steps=[('timefeaturesencoder',
                                                     TimeFeaturesEncoder(time_column='pickup_dateti...
                                    Pipeline(steps=[('direction', Direction()),
                                                    ('standardscaler',
                                                 

In [10]:
trainer.pipeline

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('distance',
                                                  Pipeline(steps=[('distancetransformer',
                                                                   DistanceTransformer(distance_type='manhattan')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['pickup_latitude',
                                                   'pickup_longitude',
                                                   'dropoff_latitude',
                                                   'dropoff_longitude']),
                                                 ('time_features',
                                                  Pipeline(steps=[('timefeaturesencoder',
                                                                   TimeFeaturesEncode...
                              interaction_constraints=None,
                              learning_rate=[0.1, 0.01, 0.05],
                              max_delta_step=None, max_depth=range(10, 20, 2),
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None,
                              n_estimators=range(60, 220, 40), n_jobs=-1,
                              num_parallel_tree=None, random_state=None,
                              reg_alpha=None, reg_lambda=None,
                              scale_pos_weight=None, subsample=None,
                              tree_method=None, validate_parameters=None,
                              verbosity=None))])

In [19]:
model_estimators = {'rgs__{}'.format(key):value for key, value in trainer.model_params.items()}

In [18]:
from sklearn.model_selection import RandomizedSearchCV

In [20]:
clf = RandomizedSearchCV(trainer.pipeline, model_estimators, scoring='rmse')
search = clf.fit(trainer.X_train, trainer.y_train)

AttributeError: 'RandomizedSearchCV' object has no attribute '_best_estimator'

In [24]:
search.best_estimator_

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('distance',
                                                  Pipeline(steps=[('distancetransformer',
                                                                   DistanceTransformer(distance_type='manhattan')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['pickup_latitude',
                                                   'pickup_longitude',
                                                   'dropoff_latitude',
                                                   'dropoff_longitude']),
                                                 ('time_features',
                                                  Pipeline(steps=[('timefeaturesencoder',
                                                                   TimeFeaturesEncode...
                              colsample_bytree=1, gamma=3, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.05,
                              max_delta_step=0, max_depth=12,
                              min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=140,
                              n_jobs=-1, num_parallel_tree=1, random_state=0,
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                              subsample=1, tree_method='exact',
                              validate_parameters=1, verbosity=None))])